In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
cd '../input/pccnethajj/PCC-Net'

In [ ]:
!pip install easydict


In [ ]:
from tensorboardX import SummaryWriter
import os
import random
import cv2
import torch
from torch import optim
from torch.autograd import Variable
from torch.nn import NLLLoss2d
from torch.optim.lr_scheduler import StepLR
import torchvision.transforms as standard_transforms
import torchvision.utils as vutils
from models.CC import CrowdCounter
from config import cfg
from loading_data import loading_data
from misc.utils import *
from misc.timer import Timer
import pdb
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
import numpy as np
from sklearn.metrics import confusion_matrix, plot_confusion_matrix,multilabel_confusion_matrix,roc_curve,roc_auc_score
import seaborn as sn


def miou(im1, im2):
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch")

    intersection = np.logical_and(im1, im2)
    union = np.logical_or(im1, im2)

    return intersection.sum() / union.sum()

def dice(im1, im2):
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch")

    intersection = np.logical_and(im1, im2)

    return 2. * intersection.sum() / (im1.sum() + im2.sum())

    
train_set, train_loader, val_set, val_loader, restore_transform = loading_data()

_t = {'iter time' : Timer(),'train time' : Timer(),'val time' : Timer()} 

rand_seed = cfg.TRAIN.SEED    
if rand_seed is not None:
    np.random.seed(rand_seed)
    torch.manual_seed(rand_seed)
    torch.cuda.manual_seed(rand_seed)

def main():

    cfg_file = open('./config.py',"r")  
    cfg_lines = cfg_file.readlines()
    
    torch.cuda.set_device(cfg.TRAIN.GPU_ID[0])
    torch.backends.cudnn.benchmark = True

    net = CrowdCounter(ce_weights=train_set.wts,modelname='vgg_backbone')

    net.train()
    
    i_tb = 0
    epoch = 1
    model_path = './exp/all_ep_299.pth'
    _t['val time'].tic()
    validate(val_loader, model_path, epoch, restore_transform)
    _t['val time'].toc(average=False)
    print( 'val time of one epoch: {:.2f}s'.format(_t['val time'].diff))


def validate(val_loader, model_path, epoch, restore):
    net = CrowdCounter(ce_weights=train_set.wts)
    net.load_state_dict(torch.load(model_path))
    net.cuda()
    net.eval()
    print( '='*50 )
    val_loss_mse = []
    val_loss_cls = []
    val_loss_seg = []
    val_loss = []
    mae = 0.0
    mse = 0.0
    count = 0
    accuracy_total = 0
    precision_total = 0
    recall_total = 0
    f1_total = 0
    total_dice = 0
    total_miou = 0
    confusion_matrix_total = None
    for vi, data in enumerate(val_loader, 0):
        img, gt_map, gt_cnt, roi, gt_roi, gt_seg = data
        with torch.no_grad():
            count+=1
            img = Variable(img).cuda()
            gt_map = Variable(gt_map).cuda()
            gt_seg = Variable(gt_seg).cuda()

            roi = Variable(roi[0]).cuda().float()
            gt_roi = Variable(gt_roi[0]).cuda()

            pred_map,pred_cls,pred_seg = net(img, gt_map, roi, gt_roi, gt_seg)
            cloned_pred = pred_seg.clone().detach().cpu().numpy()
            cloned_gt = gt_seg.clone().detach().cpu().numpy()
            
            loss1,loss2,loss3 = net.f_loss()
            val_loss_mse.append(loss1.item())
            val_loss_cls.append(loss2.item())
            val_loss_seg.append(loss3.item())
            val_loss.append(net.loss.item())
            # class_id = np.argmax(pred_cls)
            pred_map = pred_map.data.cpu().numpy()/cfg.DATA.DEN_ENLARGE
            gt_map = gt_map.data.cpu().numpy()/cfg.DATA.DEN_ENLARGE
            #print(pred_seg.shape)
            pred_seg = pred_seg.cpu().max(1)[1].squeeze_(1).data.numpy()
            gt_seg = gt_seg.data.cpu().numpy()
            gt_count = np.sum(gt_map)
            pred_cnt = np.sum(pred_map)
            total_dice = dice(gt_seg.flatten(), pred_seg.flatten())+total_dice
            total_miou = miou(gt_seg.flatten(), pred_seg.flatten())+total_miou
            # print(gt_seg.flatten().shape, pred_seg.flatten().shape)

            color = (255, 0, 0)
  
            # Line thickness of 2 px
            thickness = 2
            image = img.cpu()[0,:,:,:].permute(1, 2, 0).numpy()
            maxi = np.max(image)
            mini = np.min(image)
            image = (image-mini)*255/(maxi-mini)
            image = np.array(image, np.uint8)
            predicted_class=pred_cls
            pred_cls = pred_cls.cpu().numpy().tolist()
            roi = roi.cpu().numpy().tolist()
            gt_roi=gt_roi.cpu().numpy()
            
            predicted_class = predicted_class.cpu().numpy()
            ####
            predicted_class_score = np.zeros_like(predicted_class)
            predicted_class_score[np.arange(len(predicted_class)), predicted_class.argmax(1)] = 1    
            #######
            #print(gt_roi.flatten().tolist())
            #print(predicted_class_score.flatten().tolist())
            if count==1:
                confusion_matrix_total = confusion_matrix(gt_roi.flatten().tolist(), predicted_class_score.flatten().tolist())
            else:
                confusion_matrix_total += confusion_matrix(gt_roi.flatten().tolist(), predicted_class_score.flatten().tolist())
            #print(confusion_matrix_total.shape)
            for i in range(len(roi)):
              # start_point = (int(r[2]), int(r[1]))
              # end_point = (int(r[4]), int(r[3]))
              xmin = int(roi[i][1])
              ymin = int(roi[i][2])
              xmax = int(roi[i][3])
              ymax = int(roi[i][4])
              start_point = (xmin, ymin)
              end_point = (xmax, ymax)
              # print(start_point, end_point) 
              
              image=image.copy()
              image = cv2.rectangle(image,start_point, end_point, color, thickness)
              cls = np.argmax(pred_cls[i])
              cv2.putText(image,str(cls),(xmin,ymin),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)
            # print(type(image))
            plt.imshow((image))
           # plt.show()
            # cv2.imshow(image)
            # cv2.imwrite('image.png', image)
            # plt.imshow(img[0,:,:,:].cpu().permute(1,2,0))
            # plt.show()
            # plt.imshow(gt_seg[0,:,:])
            # plt.show()
            # plt.imshow(gt_seg[0,:,:])
            # plt.show()
            #change the gt_seg --> gt_roi and pred_seg --> pred_cls


            #print(calculate_lane_metrics(pred_cls,gt_roi.cpu().numpy().tolist(), 10))
            #print(type(gt_seg))
            #print(type(pred_seg))
            #print(type(pred_cls))
            

           # fpr, tpr, thresholds = roc_curve(gt_roi.flatten(), predicted_class_score.flatten())
           # auc = roc_auc_score(gt_roi.flatten(), predicted_class_score.flatten())
           # print('AUC: %.3f' % auc)
            accuracy = accuracy_score(gt_roi.flatten(), predicted_class_score.flatten())
            precision = precision_score(gt_roi.flatten(), predicted_class_score.flatten()) 
            f1 = f1_score(gt_roi.flatten(), predicted_class_score.flatten())
            recall = recall_score(gt_roi.flatten(), predicted_class_score.flatten())
            pred_dummy = np.array(pred_seg>0.5, np.float32)
            accuracy_total = accuracy+accuracy_total
            recall_total=recall+recall_total
            precision_total=precision+precision_total
            f1_total=f1+f1_total
            mae += abs(gt_count-pred_cnt)
            mse += ((gt_count-pred_cnt)*(gt_count-pred_cnt))
            
            
    fpr, tpr, thresholds = roc_curve(gt_roi.flatten(), predicted_class_score.flatten())
    auc = roc_auc_score(gt_roi.flatten(), predicted_class_score.flatten())
    plt.plot(fpr, tpr, linestyle='--', label='ROC')
    print('AUC: %.3f' % auc)
    plt.figure()
    confusion = confusion_matrix_total
    sn.heatmap(confusion, annot=True, annot_kws={"size": 16})
    plt.imshow(image)
    plt.show()
    mean_iou = total_miou/count*100
    mean_dice = total_dice/count*100
    mean_accuracy = accuracy_total/count*100
    mean_f1 = f1_total/count*100
    mean_recall = recall_total/count*100
    mean_precision = precision_total/count*100
    mae = mae/val_set.get_num_samples()
    mse = np.sqrt(mse/val_set.get_num_samples())
    loss1 = np.mean(val_loss_mse)
    loss2 = np.mean(val_loss_cls)
    loss3 = np.mean(val_loss_seg)
    loss = np.mean(val_loss)
    
   # fpr=[]
   # tpr=[]
   # for i in range(10):
    #    fpr[i], tpr[i],_ =roc_curve(gt_roi[:,i], predicted_class_score[:,i])
        #gt[:,i]=
 ###############################################################################   
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(10):
        print(f"class:= {i+1}")
        fpr[i], tpr[i], _ = roc_curve(gt_roi[:, i], predicted_class_score[:, i])
        plt.figure()
        plt.plot(fpr[i], tpr[i], linestyle='--', label='ROC')
        plt.title("class "+ str(i+1))
        try:
            auc=roc_auc_score(gt_roi[:, i], predicted_class_score[:, i])
            print(f"accuracy:= {auc}")
        except:
            print("No positive prediction found so auc can't be compputed")
####################################################################################   
    #print(gt_roi)
    #print( predicted_class_score)
    #print(gt_roi.shape)
    #print(gt_roi[:,1])
   # fpr, tpr, thresholds =roc_curve(gt_roi.flatten(), predicted_class_score.flatten())
    print( '='*50 )
    print( '    '+ '-'*20 )
    print( '    [mae %.1f mse %.1f], [val loss %.8f %.8f %.4f %.4f]' % (mae, mse, loss, loss1, loss2, loss3) )        
    print( '    '+ '-'*20 )
    print(f"accuracy:{mean_accuracy}\nprecision:{mean_precision}\nrecall:{mean_recall}\nf1_score:{mean_f1}")
    print(f"Confusicon_matrix:{confusion}")
    print(f'MIOU:{mean_iou}\nDice Score:{mean_dice}')
    print( '='*50 )


if __name__ == '__main__':
    main()



